In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as sps

from LinearRegression import linear_regression_loss, linear_regression_gradient
from LogisticRegression import logistic_regression_loss, logistic_regression_gradient

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from functools import partial

In [2]:
import importlib
import methods
importlib.reload(methods)
from methods import MinibatchSGD, AsynchronousGD, RennalaSGD, WorkerState

In [3]:
from sklearn.datasets import load_diabetes
diabetes_X, diabetes_y = load_diabetes(return_X_y=True)
diabetes_X = np.hstack((diabetes_X, np.ones((len(diabetes_X), 1))))

In [4]:
n_workers = 4
time_distributions = [sps.norm(i+1, scale=1) for i in range(n_workers)]
batch_sizes = np.arange(1, n_workers + 1) * 10


n_iterations = 2_000

w = np.zeros(diabetes_X.shape[1])
lr = 0.01

In [ ]:
range_of_k = np.array(np.array([.01, .05, .1, .2]) * len(diabetes_X), dtype='int')
compressions = ['none', 
                'randk',
                'topk',
                ]
range_of_k

array([ 4, 22, 44, 88])

In [ ]:
from IPython.display import clear_output

times = dict()
losses = dict()

for compression in compressions:
    for compression_rate in range_of_k:
        minibatch_sgd = MinibatchSGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                                    [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                    learning_rate=lr, \
                                    compression_flag=compression, compression_size=compression_rate)
        async_gd = AsynchronousGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                                    [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                    learning_rate=lr, \
                                    compression_flag=compression, compression_size=compression_rate)
        rennala_sgd = RennalaSGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                                    [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                    learning_rate=lr, \
                                    compression_flag=compression, compression_size=compression_rate)
        rennala_sgd.set_batch_size(5)

        methods = {'Minibatch SGD': minibatch_sgd,
                   'Async GD': async_gd,
                   #'Rennala SGD': rennala_sgd
                   }

        for method_name in methods:
            if method_name not in times:
                times[method_name] = dict()
            if method_name not in losses:
                losses[method_name] = dict()
            if compression not in times[method_name]:
                times[method_name][compression] = dict()
            if compression not in losses[method_name]:
                losses[method_name][compression] = dict()

            w = np.zeros(diabetes_X.shape[1])
            print(f"Running {method_name} with compression {compression} and compression rate {compression_rate}")
            current_x, loss_history, computation_times, x_history = methods[method_name].run_steps(n_iterations)
            times[method_name][compression][compression_rate] = computation_times
            losses[method_name][compression][compression_rate] = loss_history

            # Clear output every 5 iterations
            if (compression_rate + 1) % 5 == 0:
                clear_output(wait=True)
                print(f"Cleared output after {compression_rate + 1} iterations")

Running Minibatch SGD with compression randk and compression rate 4


: 